In [8]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.6'
spark_version = 'spark-2.4.7'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (112 kB/s)
Reading package lists... Done


In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-09-27 23:37:23--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.99MB/s    in 0.2s    

2020-09-27 23:37:23 (4.99 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Import struct fields that we can use
from pyspark.sql.types import *
# Read in data from files
from pyspark import SparkFiles

In [ ]:
# Next we need to create the list of struct fields
schema = [StructField("aisle_id", IntegerType(), True),
          StructField("aisle", StringType(), True),]
# Pass in our fields
final = StructType(fields=schema)
# Read our data with our new schema

filepath_aisles="/content/aisles.csv"
aisles_df = spark.read.csv(filepath_aisles, sep=",", header=True, schema=final)

In [ ]:
# Next we need to create the list of struct fields
schema = [StructField("department_id", IntegerType(), True),
          StructField("department", StringType(), True),]
# Pass in our fields
final = StructType(fields=schema)
# Read our data with our new schema

filepath_department="/content/departments.csv"
department_df = spark.read.csv(SparkFiles.get(filepath_department), sep=",", header=True, schema=final)

In [9]:
# Next we need to create the list of struct fields
schema = [StructField("product_id", IntegerType(), True),
          StructField("product_name", StringType(), True),
          StructField("aisle_id", IntegerType(), True),
          StructField("department_id", IntegerType(), True),]
# Pass in our fields
final = StructType(fields=schema)
# Read our data with our new schema

filepath_products="/content/products.csv"
products_df = spark.read.csv(SparkFiles.get(filepath_products),schema=final, sep=",", header=True, )

In [ ]:
# Next we need to create the list of struct fields
schema = [StructField("order_id", IntegerType(), True),
          StructField("user_id", IntegerType(), True),
          StructField("eval_set", StringType(), True),
          StructField("order_number", IntegerType(), True),
          StructField("order_dow", IntegerType(), True),
          StructField("order_hour_of_day", IntegerType(), True),
          StructField("days_since_prior_order", FloatType(), True),]
# Pass in our fields
final = StructType(fields=schema)
# Read our data with our new schema

filepath_orders="/content/orders.csv"
orders_df = spark.read.csv(SparkFiles.get(filepath_orders), schema=final, sep=",", header=True)

In [17]:
# Next we need to create the list of struct fields
schema = [StructField("order_id", IntegerType(), True),
          StructField("product_id", IntegerType(), True),
          StructField("add_to_cart_order", IntegerType(), True),
          StructField("reordered", IntegerType(), True),]
# Pass in our fields
final = StructType(fields=schema)
# Read our data with our new schema

filepath_orders_prior="/content/order_products__prior.csv"
orders_prior_df = spark.read.csv(SparkFiles.get(filepath_orders_prior),schema=final, sep=",", header=True)

In [15]:
# Next we need to create the list of struct fields
schema = [StructField("order_id", IntegerType(), True),
          StructField("product_id", IntegerType(), True),
          StructField("add_to_cart_order", IntegerType(), True),
          StructField("reordered", IntegerType(), True),]
# Pass in our fields
final = StructType(fields=schema)
# Read our data with our new schema

filepath_orders_train="/content/order_products__train.csv"
orders_train_df = spark.read.csv(SparkFiles.get(filepath_orders_train), sep=",", header=True,schema=final)

In [10]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://finalprojectdb.cyj4dabyex0o.us-east-2.rds.amazonaws.com:5432/final_project_db"
config = {"user":"root", 
          "password": "FinalProject_2020", 
          "driver":"org.postgresql.Driver"}

In [ ]:
aisles_df.write.jdbc(url=jdbc_url, table='aisle_tbl', mode=mode, properties=config)

In [ ]:
department_df.write.jdbc(url=jdbc_url, table='department_tbl', mode=mode, properties=config)

In [12]:
products_df = products_df.na.drop(subset=["product_id"])
products_df = products_df.na.drop(subset=["aisle_id"])
products_df = products_df.na.drop(subset=["department_id"])


In [13]:
products_df.write.jdbc(url=jdbc_url, table='product_tbl', mode=mode, properties=config)

In [ ]:
orders_df = orders_df.na.drop(subset=["order_id"])
orders_df = orders_df.na.drop(subset=["user_id"])

In [ ]:
orders_df.write.jdbc(url=jdbc_url, table='orders_tbl', mode=mode, properties=config)

In [19]:
orders_prior_df.write.jdbc(url=jdbc_url, table='orders_product_prior', mode=mode, properties=config)

KeyboardInterrupt: ignored

In [ ]:
orders_train_df = orders_train_df.na.drop(subset=["order_id"])
orders_train_df = orders_train_df.na.drop(subset=["product_id"])

In [16]:
orders_train_df.write.jdbc(url=jdbc_url, table='orders_product_train', mode=mode, properties=config)